In [1]:
import numpy as np
import pandas
import sys
import dill
import os
import matplotlib.pyplot as plt
import symusic

In [2]:
sys.path.insert(1, '/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/')

In [3]:
from benchmark_generators import CadenceGenerator, IntervalLikelihoodGenerator, \
                                IntervalRecognitionGenerator, MelodyContinuationGenerator, \
                                ScaleFillingGenerator, BarShufflingGenerator, TrackTranspositionGenerator, \
                                RandomNoteAlterationGenerator

from utils import time_dilate_midi, transpose_midi
#from utils import reshuffle_bars, transpose_track

In [4]:
from pathlib import Path
from amads.expectation.tokenizer import MelodyIntervalTokenizer, IOITokenizer, Token, REMITokenizer, TSDTokenizer, TSDTokenizer_Custom
from amads.expectation.dataset import ScoreDataset
from amads.expectation.model import LSTM
from amads.expectation.metrics import NegativeLogLikelihood, Entropy, InformationContent
from amads.core.basics import Note, Score
from scipy import stats
import torch
from collections import defaultdict

import pandas as pd
import seaborn as sns

In [5]:
# # INTERVAL PREDICTION

# intervals = ['-octave',
#  '-maj7',
#  '-min7',
#  '-maj6',
#  '-min6',
#  '-p5',
#  '-tritone',
#  '-p4',
#  '-maj3',
#  '-min3',
#  '-maj2',
#  '-min2',
#  'unison',
#  'min2',
#  'maj2',
#  'min3',
#  'maj3',
#  'p4',
#  'tritone',
#  'p5',
#  'min6',
#  'maj6',
#  'min7',
#  'maj7',
#  'octave']

# n_perm = 5

# for i in range(n_perm):

#     generator = IntervalRecognitionGenerator(output_dir = '/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/IDyOM_data/interval_recognition_'+str(i + 1)+'/')

#     for interval in intervals:

#         tests = generator.generate_comprehensive_test(interval, num_examples=10)

In [21]:
PATH_TO_BENCHMARK_SUITE = '/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/benchmark_suite/'


In [78]:
lstm = torch.load('../lstm_atepp_not_bad_quality_embedd23_hidden64_300epochs_batch4_lr0.01_tokenizer_fixed.pth', map_location='cpu')
lstm.eval()
lstm.device = 'cpu'
model = lstm

custom_params = {'time_range' : (0.01, 1), 'time_factor': 1}
tokenizer = TSDTokenizer_Custom(config_params = custom_params)

# lstm_small = torch.load('../lstm_model_test_3.pth')
# lstm_small.eval()


In [8]:
def get_mean_nll(midi_path) -> float:

    # Tokenize both MIDI files
    tokens = tokenizer.tokenize(midi_path)
    
    # Get model predictions
    predictions = model.predict_sequence(tokens)
    
    # Calculate mean NLLs (ignoring None values)
    nlls = [nll for nll in predictions.nlls if nll is not None]
    
    mean_nll = np.mean(nlls)
    
    # Return results
    return mean_nll

In [61]:
def get_total_nll(midi_path) -> float:

    # Tokenize both MIDI files
    tokens = tokenizer.tokenize(midi_path)
    
    # Get model predictions
    predictions = model.predict_sequence(tokens)
    
    # Calculate mean NLLs (ignoring None values)
    nlls = [nll for nll in predictions.nlls if nll is not None]
    
    total_nll = np.sum(nlls)
    
    # Return results
    return total_nll

In [9]:

#tokenizer = TSDTokenizer()

In [10]:
# CADENCE PREDICTION

In [ ]:
# generator = CadenceGenerator(output_dir='/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/examples/cadence_examples/')
# cadence_resolutions = generator.generate_all_resolutions(key_name='C', backbone_progression = [1,4,1,5])

In [11]:
def run_cadence_prediction_benchmark(get_mean_nll, PATH_TO_BENCHMARK_SUITE, return_all_results = False):
    
    '''
    Runs cadence prediction benchmark.
    Arguments:
    get_mean_nll: callable, str -> float, takes in midi file and outputs mean NLL
    PATH_TO_BENCHMARK_SUITE: str, path to benchmark_suite directory
    return_all_results: if True, return the percentile of the tonic key for every studied key. If False, just return the mean. Default=False
    
    Outputs:
    results: dataframe of percetile of tonic cadence for each of 12 tonic keys (C4-B5).
    mean_percentile: Average percentile of tonic across all keys. Higher = better.
    
    '''
    
    all_maj = []
    all_min = []
    all_dim = []

    for i in range(12):

        maj_nll = []
        min_nll = []
        dim_nll = []

        for j in range(12):

            major = PATH_TO_BENCHMARK_SUITE + 'cadence/' + str(60+i)+'_from_I-IV-I-V_to_' + str(j)+'_maj.mid'
            minor = PATH_TO_BENCHMARK_SUITE + 'cadence/' + str(60+i)+'_from_I-IV-I-V_to_' + str(j)+'_min.mid'
            dimin = PATH_TO_BENCHMARK_SUITE + 'cadence/' + str(60+i)+'_from_I-IV-I-V_to_' + str(j)+'_dim.mid'

            maj_nll.append(get_mean_nll(major))
            min_nll.append(get_mean_nll(minor))
            dim_nll.append(get_mean_nll(dimin)) 

        all_maj.append(maj_nll)
        all_min.append(min_nll)
        all_dim.append(dim_nll)
        
    all_maj = pd.DataFrame(all_maj, columns = [str(x) + '_maj' for x in range(0,12)]).T
    all_min = pd.DataFrame(all_min, columns = [str(x) + '_min' for x in range(0,12)]).T
    all_dim = pd.DataFrame(all_dim, columns = [str(x) + '_dim' for x in range(0,12)]).T
    
    all_results = pd.concat([all_maj, all_min, all_dim])
    all_results = (-all_results).rank(0, pct = True).T[['0_maj']].rename(columns = {'0_maj':'Percentile of tonic'})
    mean_percentile = np.mean(all_results)
    
    if return_all_results == True:
        return all_results, mean_percentile
    
    else:
        return mean_percentile

In [12]:
run_cadence_prediction_benchmark(get_mean_nll, PATH_TO_BENCHMARK_SUITE, return_all_results=False)

0.8958333333333334

In [13]:
# SCALE FILLING

In [ ]:
# generator = ScaleFillingGenerator(output_dir="/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/benchmark_suite/scale_filling/")

# c_major_context, control_files = generator.generate_all_scale_variations(
#     key_name="C",
#     mode='major', include_controls=True
# )

# for k, fname in enumerate(c_major_context):
#     for i in range(-12,13):
#         new_fname = str(60+i) + fname.split('/')[-1][1:].split('.')[0] + '.mid'
#         new_fname_control = str(60+i) + fname.split('/')[-1][1:].split('.')[0] + '_control.mid'
        
#         transpose_midi(input_path=fname, output_path=PATH_TO_BENCHMARK_SUITE + 'scale_filling/' + new_fname, semitones = i)
#         transpose_midi(input_path=control_files[k], output_path= PATH_TO_BENCHMARK_SUITE + 'scale_filling/' + new_fname_control, semitones = i)

In [76]:
scale_filling_dir = '/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/benchmark_suite/scale_filling/'
files = [scale_filling_dir + x for x in os.listdir(scale_filling_dir)]

In [77]:
correct_file = [x for x in files if ('correct' in x) and ('temp' not in x)][0]
incorrect_files = [x for x in files if (x.endswith('mid')) and ('correct' not in x) and ('temp' not in x)]

In [25]:
import glob

In [27]:
PATH_TO_BENCHMARK_SUITE

'/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/benchmark_suite/'

In [129]:
def run_scale_filling_benchmark(get_total_nll, PATH_TO_BENCHMARK_SUITE, return_all_results = False):
    
    '''
    Runs scale filling benchmark.
    Arguments:
    get_total_nll: callable, str -> float, takes in midi file and outputs total NLL
    PATH_TO_BENCHMARK_SUITE: str, path to benchmark_suite directory
    return_all_results: if True, return the percentile of the tonic key for every studied key. If False, just return the mean. Default=False
    
    Outputs:
    results: dataframe of percetile of tonic cadence for each of 12 tonic keys (C4-B5).
    mean_percentile: Average percentile of tonic across all keys. Higher = better.
    
    '''
    
    all_correct_nlls = []
    all_incorrect_nlls = []

    for i in range(48,73):

        all_files = glob.glob(PATH_TO_BENCHMARK_SUITE + 'scale_filling/'+str(i) + '_*')
        correct = [x for x in all_files if (x.endswith('mid')) and ('correct' in x) and ('control' not in x)][0]
        correct_control = correct.split('.mid')[0] + '_control.mid'

        wrong = [x for x in all_files if (x.endswith('mid')) and ('correct' not in x) and ('control' not in x)]
        wrong_control = [x.split('.mid')[0] + '_control.mid' for x in wrong]

        #THIS VERSION INCLUDES A NORMALIZATION
        nll_correct = get_total_nll(correct) - get_total_nll(correct_control)
        nlls_incorrect = [get_total_nll(wrong[k]) - get_total_nll(wrong_control[k]) for k in range(len(wrong))]
        #nlls_incorrect = [get_mean_nll(wrong[k]) for k in range(len(wrong))]

        all_correct_nlls.append(nll_correct)
        all_incorrect_nlls.append(nlls_incorrect)

    correct_scale_rank = [np.argsort(np.argsort(all_incorrect_nlls[i] + [all_correct_nlls[i]]))[-1] + 1  for i in range(-12,13)]
    correct_scale_rank = pd.DataFrame(correct_scale_rank, columns = ['Correct Scale Rank'])

    all_results = pd.DataFrame((((24 - correct_scale_rank) + 1)/24).values, columns = ['Average Percentile'])
    mean_percentile = np.mean(all_results)
    
    if return_all_results == True:
        return all_results, mean_percentile
    
    else:
        return mean_percentile

In [130]:
run_scale_filling_benchmark(get_total_nll, PATH_TO_BENCHMARK_SUITE, return_all_results=False)

0.7283333333333334

In [120]:
# 

In [ ]:
generator = ScaleFillingGenerator(output_dir="/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/examples/scale_filling_examples/")

c_major_context = generator.generate_all_scale_variations(
    key_name="C",
    mode='major',
)

In [19]:
scale_filling_dir = '/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/examples/scale_filling_examples/'
files = [scale_filling_dir + x for x in os.listdir(scale_filling_dir)]
correct_file = [x for x in files if ('correct' in x) and ('temp' not in x)][0]
incorrect_files = [x for x in files if (x.endswith('mid')) and ('correct' not in x) and ('temp' not in x)]

all_correct_nlls = []
all_incorrect_nlls = []

for i in range(-12,13):
    print(i)
    correct = transpose_midi(input_path=correct_file, output_path='/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/examples/scale_filling_examples/temp_correct.mid', semitones = i)
    wrong = [transpose_midi(input_path=x, output_path='/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/examples/scale_filling_examples/temp_wrong'+str(j)+'.mid', semitones = i) for j, x in enumerate(incorrect_files)]
    nll_correct = get_mean_nll(model, tokenizer, correct)#compare_midi_sequence_nlls(lstm_small, TSDTokenizer(), correct, wrong[0])['correct_mean_nll']
    nlls_incorrect = [get_mean_nll(model, tokenizer, x) for x in wrong]

    all_correct_nlls.append(nll_correct)
    all_incorrect_nlls.append(nlls_incorrect)
    
# tokenizer = TSDTokenizer()
# correct_midi_path = os.path.listdir(scale_filling_dir+'*correct*')
# correct_tokens = tokenizer.tokenize(correct_midi_path)
# incorrect_tokens = tokenizer.tokenize(incorrect_midi_path)


-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
0
1
2
3
4
5
6
7
8
9
10
11
12


In [20]:
correct_scale_rank = [np.argsort(np.argsort(all_incorrect_nlls[i] + [all_correct_nlls[i]]))[-1] + 1  for i in range(-12,13)]

In [21]:
correct_scale_rank = pd.DataFrame(correct_scale_rank, columns = ['Correct Scale Rank'])

In [131]:
# Interval recognition

In [133]:
intervals = ['-octave',
 '-maj7',
 '-min7',
 '-maj6',
 '-min6',
 '-p5',
 '-tritone',
 '-p4',
 '-maj3',
 '-min3',
 '-maj2',
 '-min2',
 'unison',
 'min2',
 'maj2',
 'min3',
 'maj3',
 'p4',
 'tritone',
 'p5',
 'min6',
 'maj6',
 'min7',
 'maj7',
 'octave']

In [134]:
# for perm in range(1,6):
    
#     generator = IntervalRecognitionGenerator(output_dir = '/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/benchmark_suite/interval/perm' + str(perm) + '/')

#     final_results = defaultdict(list)

#     for interval in intervals:

#         results = {key: [] for key in intervals}

#         tests = generator.generate_comprehensive_test(interval, num_examples=4)
        

-octave
-maj7
-min7
-maj6
-min6
-p5
-tritone
-p4
-maj3
-min3
-maj2
-min2
unison
min2
maj2
min3
maj3
p4
tritone
p5
min6
maj6
min7
maj7
octave
-octave
-maj7
-min7
-maj6
-min6
-p5
-tritone
-p4
-maj3
-min3
-maj2
-min2
unison
min2
maj2
min3
maj3
p4
tritone
p5
min6
maj6
min7
maj7
octave
-octave
-maj7
-min7
-maj6
-min6
-p5
-tritone
-p4
-maj3
-min3
-maj2
-min2
unison
min2
maj2
min3
maj3
p4
tritone
p5
min6
maj6
min7
maj7
octave
-octave
-maj7
-min7
-maj6
-min6
-p5
-tritone
-p4
-maj3
-min3
-maj2
-min2
unison
min2
maj2
min3
maj3
p4
tritone
p5
min6
maj6
min7
maj7
octave
-octave
-maj7
-min7
-maj6
-min6
-p5
-tritone
-p4
-maj3
-min3
-maj2
-min2
unison
min2
maj2
min3
maj3
p4
tritone
p5
min6
maj6
min7
maj7
octave


In [176]:
def run_interval_recognition_benchmark(get_mean_nll, PATH_TO_BENCHMARK_SUITE, return_all_results = False, n_perm=5):
    
    '''
    Runs interval recognition benchmark.
    Arguments:
    get_mean_nll: callable, str -> float, takes in midi file and outputs mean NLL
    PATH_TO_BENCHMARK_SUITE: str, path to benchmark_suite directory
    return_all_results: if True, return the percentile of the tested for every interval and permutation. If False, take the average over permutations and intervals and just return this value. Default=False
    n_perm: set the number of times the experiment is repeated and averaged over. Must be >1. 
    
    Outputs:
    results: dataframe of percentile of the tested interval for each interval (-octave to octave) for each permutation.
    mean_percentile: Average percentile of tonic across all keys. Higher = better.
    
    '''
    
    intervals = ['-octave',
     '-maj7',
     '-min7',
     '-maj6',
     '-min6',
     '-p5',
     '-tritone',
     '-p4',
     '-maj3',
     '-min3',
     '-maj2',
     '-min2',
     'unison',
     'min2',
     'maj2',
     'min3',
     'maj3',
     'p4',
     'tritone',
     'p5',
     'min6',
     'maj6',
     'min7',
     'maj7',
     'octave']

    final_results = defaultdict(list)

    n_perm = n_perm

    print("Testing interval...")
    for interval in intervals:

        print(interval)
        results = {key: [] for key in intervals}

        for i in range(1, n_perm+1):
            #print(i)
            tests = glob.glob(PATH_TO_BENCHMARK_SUITE + 'interval/perm' + str(i) + '/' + interval + '*')
            test_intervals = [x.split('.mid')[0].split('_')[-1] for x in tests]
            tests_dict = dict(zip(test_intervals, tests))

            keys = []
            nlls = []

            for key, value in tests_dict.items():
                keys.append(key)
                nlls.append(get_mean_nll(value))

            min_nll = np.min(nlls)
            nlls = np.array(nlls) - min_nll#

            for j, x in enumerate(keys):
                if x == 'correct':
                    results[interval].append(nlls[j])
                else:
                    results[x].append(nlls[j])

        target_relative_nlls = pd.DataFrame(results).T.rank(axis=0).T[interval]

        final_results[interval] = list(target_relative_nlls.values.flatten())

    final_results_df = pd.DataFrame(final_results, index = ['Permutation ' + str(x) for x in range(1,n_perm+1)])#.melt(var_name='Interval', value_name = 'Rank')['Rank'].mean()
    final_results_df = ((24 - final_results_df) + 1)/24

    mean_percentile = final_results_df.mean().mean()

    
    if return_all_results == True:
        return final_results_df, mean_percentile
    
    else:
        return mean_percentile

In [173]:
res, mean_percentile = run_interval_recognition_benchmark(get_mean_nll, PATH_TO_BENCHMARK_SUITE=PATH_TO_BENCHMARK_SUITE, return_all_results=True)


Testing interval...
-octave
-maj7
-min7
-maj6
-min6
-p5
-tritone
-p4
-maj3
-min3
-maj2
-min2
unison
min2
maj2
min3
maj3
p4
tritone
p5
min6
maj6
min7
maj7
octave


In [24]:
# INTERVAL PREDICTION

generator = IntervalRecognitionGenerator(output_dir = '/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/examples/interval_examples/')

final_results = defaultdict(list)

for interval in intervals:
    
    print(interval)
    results = {key: [] for key in intervals}

    n_perm = 2

    for i in range(n_perm):

        # Generate comprehensive test for interval
        tests = generator.generate_comprehensive_test(interval, num_examples=10)

        tokenizer = TSDTokenizer()
        
        keys = []
        nlls = []
        
        for key, value in tests.items():
            keys.append(key)
            nlls.append(get_mean_nll(lstm, TSDTokenizer(), value))
            
        min_nll = np.min(nlls)
        nlls = np.array(nlls) - min_nll#

        for j, x in enumerate(keys):
            results[x].append(nlls[j])

    target_relative_nlls = pd.DataFrame(results).T.rank(axis=0).T[interval]
        
    final_results[interval] = list(target_relative_nlls.values.flatten())

-octave
-maj7
-min7
-maj6
-min6
-p5
-tritone
-p4
-maj3
-min3
-maj2
-min2
unison
min2
maj2
min3
maj3
p4
tritone
p5
min6
maj6
min7
maj7
octave


In [175]:
#TRANSPOSITION INVARIANCE

In [178]:
# # Create the generator
# generator = IntervalLikelihoodGenerator(output_dir='/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/benchmark_suite/transposition/')

# # Generate all intervals from C4 (MIDI note 60)
# c4_tests = generator.generate_all_intervals(start_note=60)
# print(f"Generated {len(c4_tests)} interval tests from C4")

# notes = list(range(60-24, 60+25))
# results = {key: [] for key in notes}

# for note in notes:
#     # Generate comprehensive test for perfect fifth
#     tests = generator.generate_all_intervals(start_note=note)


In [190]:
def get_transposition_invariance(get_mean_nll, PATH_TO_BENCHMARK_SUITE, return_all_results = False):
    
    '''
    Runs transpostion invariance benchmark.
    Arguments:
    get_mean_nll: callable, str -> float, takes in midi file and outputs mean NLL
    PATH_TO_BENCHMARK_SUITE: str, path to benchmark_suite directory
    return_all_results: if True, return the likelihood of all intervals for every starting note two octaves above and below middle C. If False, just return the mean of the correlation between likelihood vectors of adjacent notes. Default=False
    
    Outputs:
    results: dataframe of the likelihood of all intervals for every starting note two octaves above and below middle C. 
    mean_percentile: Average correlation between interval likelihood vectors for adjacent notes.
    
    '''

    notes = list(range(60-24, 60+25))
    results = {key: [] for key in notes}

    intervals = ['neg_octave',
     'neg_maj7',
     'neg_min7',
     'neg_maj6',
     'neg_min6',
     'neg_p5',
     'neg_tritone',
     'neg_p4',
     'neg_maj3',
     'neg_min3',
     'neg_maj2',
     'neg_min2',
     'unison',
     'min2',
     'maj2',
     'min3',
     'maj3',
     'p4',
     'tritone',
     'p5',
     'min6',
     'maj6',
     'min7',
     'maj7',
     'octave']

    for note in notes:

        for interval in intervals:
            fname = PATH_TO_BENCHMARK_SUITE + 'transposition/' + 'pitch' + str(note) + '_' + interval + '.mid'
            results[note].append(get_mean_nll(fname))
            
    all_results = pd.DataFrame(results)

    #transposition_invariance = all_results.corr().values[np.triu_indices(24, k= 1)].mean()

    ix_adjacent_notes = np.vstack((np.arange(0,len(notes)-1),np.arange(1,len(notes))))#[:-1,:]
    ix_adjacent_notes = tuple(ix_adjacent_notes)     
    
    invariance = all_results.corr().values[ix_adjacent_notes]
    
    
    if return_all_results == True:
        return all_results, invariance
    
    else:
        return np.mean(invariance)

In [191]:
get_transposition_invariance(get_mean_nll, PATH_TO_BENCHMARK_SUITE)

0.519195675494432

In [192]:
# Create the generator
generator = IntervalLikelihoodGenerator(output_dir='/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/benchmark_suite/transposition/')

# Generate all intervals from C4 (MIDI note 60)
c4_tests = generator.generate_all_intervals(start_note=60)
print(f"Generated {len(c4_tests)} interval tests from C4")

notes = list(range(60-24, 60+25))
results = {key: [] for key in notes}

for note in notes:
    print(note)
    # Generate comprehensive test for perfect fifth
    tests = generator.generate_all_intervals(start_note=note)

    tokenizer = TSDTokenizer()

    # Tokenize both MIDI files
    tokens = [(key, tokenizer.tokenize(time_dilate_midi(input_path=value, output_path = value, scale_factor=1))) for key, value in tests.items()]
    #tokens = [(key, tokenizer.tokenize(value)) for key, value in tests.items()]

#     # Get model predictions
#     predictions = [(key, lstm_small.predict_sequence(x)) for (key, x) in tokens]
    
#     for (key, x) in predictions:
#         results[note].append(np.mean([nll for nll in x.nlls if nll is not None]))

    for key, value in tests.items():
        results[note].append(get_mean_nll(value))


In [37]:
all_results = pd.DataFrame(results)

#transposition_invariance = all_results.corr().values[np.triu_indices(24, k= 1)].mean()

ix_adjacent_notes = np.vstack((np.arange(0,len(notes)-1),np.arange(1,len(notes))))#[:-1,:]
ix_adjacent_notes = tuple(ix_adjacent_notes)

In [38]:
invariance = all_results.corr().values[ix_adjacent_notes]

In [39]:
np.median(invariance)

0.5459220773146873

In [237]:
def human_melody_comparison(get_mean_nll, PATH_TO_BENCHMARK_SUITE, human_ranking_data_loc = None, return_all_results = False):
        
    '''
    Runs human melodic alignment benchmark.
    Arguments:
    get_mean_nll: callable, str -> float, takes in midi file and outputs mean NLL
    PATH_TO_BENCHMARK_SUITE: str, path to benchmark_suite directory
    human_ranking_data_loc: location of Lokyan_t_01_human_processed_ratings.csv, with the human ratings data. 
    return_all_results: if True, return the correlation between model and human ratings for every context interval and for both keys (C and F#). Default=False
    
    Outputs:
    results: dataframe of the correlation between model and human ratings for every context interval and for both keys. 
    mean_correlation: Average correlation between model and human ratings across all contexts and both keys. 
    
    '''
    
    if human_ranking_data_loc==None:
        human_ranking_data_loc = PATH_TO_BENCHMARK_SUITE + '/melody_continuation/Lokyan_t_01_human_processed_ratings.csv'
        
    context_intervals = ['M2_a', 'M2_d', 'M6_a', 'M6_d', 'm3_a', 'm3_d', 'm7_a', 'm7_d']
    ending_notes = ["C4",'F#4']
        #f_sharp_results = pd.DataFrame(index = range(1,26), columns = context_intervals)
    c_results = pd.DataFrame(index = range(1,26), columns = context_intervals)

    for interval in context_intervals:

        results = {key: [] for key in range(1,26)}

        for i in range(1,26):
            fname = glob.glob(PATH_TO_BENCHMARK_SUITE + '/melody_continuation/' + interval + '_from_C4_cont_'+ str(i) + '_*.mid')[0]
            results[i].append(get_mean_nll(fname))#np.mean([nll for nll in x.nlls if nll is not None]))

        results = pd.DataFrame(results, index = [interval]).T

        c_results[interval] = results[interval]


    f_sharp_results = pd.DataFrame(index = range(1,26), columns = context_intervals)

    for interval in context_intervals:

        results = {key: [] for key in range(1,26)}
        
        for i in range(1,26):
            fname = glob.glob(PATH_TO_BENCHMARK_SUITE + '/melody_continuation/' + interval + '_from_F#4_cont_'+ str(i) + '_*.mid')[0]
            results[i].append(get_mean_nll(fname))#np.mean([nll for nll in x.nlls if nll is not None]))

        results = pd.DataFrame(results, index = [interval]).T

        f_sharp_results[interval] = results[interval]
        
    human_results = pd.read_csv(human_ranking_data_loc, index_col = 0)
    human_results_reformatted = pd.DataFrame(index = range(1,26), columns = context_intervals)
    for x, dat in human_results.groupby('stim_identity'):
        human_results_reformatted[x] = dat['mean_rating'].values

    model_human_corrs = pd.DataFrame(index = context_intervals, columns = ending_notes  + ['average'])

    c_results_norm = (c_results-c_results.mean())/c_results.std()
    f_sharp_results_norm = (f_sharp_results-f_sharp_results.mean())/f_sharp_results.std()

    for interval in context_intervals:
        model_human_corrs.at[interval, 'C4'] = stats.spearmanr(c_results[interval], -human_results_reformatted[interval])[0]
        model_human_corrs.at[interval, 'F#4'] = stats.spearmanr(f_sharp_results[interval], -human_results_reformatted[interval])[0]
        model_human_corrs.at[interval, 'average'] = stats.spearmanr(((c_results_norm + f_sharp_results_norm)/2)[interval], -human_results_reformatted[interval])[0]
        
    mean_correlation = np.mean(model_human_corrs['average'])
    
    if return_all_results == True:
        return model_human_corrs, mean_correlation
    
    else:
        return mean_correlation


In [238]:
human_melody_comparison(get_mean_nll, PATH_TO_BENCHMARK_SUITE, return_all_results=False)

0.7545058303704535

In [ ]:
/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/benchmark_suite/../human_data

In [194]:
PATH_TO_BENCHMARK_SUITE + '../human_data'

'/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/benchmark_suite/../human_data'

In [50]:
# HUMAN T01 - MELODY CONTINUATION
generator = MelodyContinuationGenerator(output_dir='/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/examples/melody_continuation_examples/')
context_dir = '/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/human_data/Stimuli/t_01/'
ending_notes = ['C4','F#4']

In [15]:
context_intervals = ['M2_a', 'M2_d', 'M6_a', 'M6_d', 'm3_a', 'm3_d', 'm7_a', 'm7_d']

#f_sharp_results = pd.DataFrame(index = range(1,26), columns = context_intervals)
c_results = pd.DataFrame(index = range(1,26), columns = context_intervals)

for interval in context_intervals:
    print(interval)
    continuations = generator.generate_continuations(context_midi_path=context_dir + 'C4/' + interval + '.mid', context_end_note=ending_notes[0], last_note_duration=480//2)

    results = {key: [] for key in range(1,26)}
#     tokenizer = TSDTokenizer()

#     # Tokenize both MIDI files
#     tokens = [(key, tokenizer.tokenize(value)) for key, value in continuations.items()]
#     #tokens = [(key, tokenizer.tokenize(value)) for key, value in tests.items()]

#     # Get model predictions
#     predictions = [(key, lstm.predict_sequence(x)) for (key, x) in tokens]

    for  key, value in continuations.items():
        results[key].append(get_mean_nll(lstm, TSDTokenizer(), value))#np.mean([nll for nll in x.nlls if nll is not None]))

    results = pd.DataFrame(results, index = [interval]).T
    
    c_results[interval] = results[interval]
    
    
f_sharp_results = pd.DataFrame(index = range(1,26), columns = context_intervals)

for interval in context_intervals:
    print(interval)
    continuations = generator.generate_continuations(context_midi_path=context_dir + 'F_sharp_4/' + interval + '.mid', context_end_note=ending_notes[1], last_note_duration=480//2)

    results = {key: [] for key in range(1,26)}
#     tokenizer = TSDTokenizer()

#     # Tokenize both MIDI files
#     tokens = [(key, tokenizer.tokenize(value)) for key, value in continuations.items()]
#     #tokens = [(key, tokenizer.tokenize(value)) for key, value in tests.items()]

#     # Get model predictions
#     predictions = [(key, lstm.predict_sequence(x)) for (key, x) in tokens]

    for  key, value in continuations.items():
        results[key].append(get_mean_nll(lstm, TSDTokenizer(), value))

    results = pd.DataFrame(results, index = [interval]).T
    
    f_sharp_results[interval] = results[interval]

M2_a
M2_d
M6_a
M6_d
m3_a
m3_d
m7_a
m7_d
M2_a
M2_d
M6_a
M6_d
m3_a
m3_d
m7_a
m7_d


In [28]:
human_results = pd.read_csv('/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/human_data/Lokyan_t_01_human_processed_ratings.csv', index_col = 0)
human_results_reformatted = pd.DataFrame(index = range(1,26), columns = context_intervals)
for x, dat in human_results.groupby('stim_identity'):
    human_results_reformatted[x] = dat['mean_rating'].values
    

In [29]:
model_human_corrs = pd.DataFrame(index = context_intervals, columns = ending_notes  + ['average'])

c_results_norm = (c_results-c_results.mean())/c_results.std()
f_sharp_results_norm = (f_sharp_results-f_sharp_results.mean())/f_sharp_results.std()

for interval in context_intervals:
    model_human_corrs.at[interval, 'C4'] = stats.spearmanr(c_results[interval], -human_results_reformatted[interval])[0]
    model_human_corrs.at[interval, 'F#4'] = stats.spearmanr(f_sharp_results[interval], -human_results_reformatted[interval])[0]
    model_human_corrs.at[interval, 'average'] = stats.spearmanr(((c_results_norm + f_sharp_results_norm)/2)[interval], -human_results_reformatted[interval])[0]

In [241]:
def human_chord_comparison(get_mean_nll, PATH_TO_BENCHMARK_SUITE, human_ranking_data_loc = None, return_all_results = False):
        
    '''
    Runs human chord alignment benchmark prediction benchmark. The stimuli were from SOURCE (Lokyan). 
    Arguments:
    get_mean_nll: callable, str -> float, takes in midi file and outputs mean NLL
    PATH_TO_BENCHMARK_SUITE: str, path to benchmark_suite directory
    human_ranking_data_loc: location of Lokyan_t_03_human_processed_ratings.csv, with the human ratings data. 
    return_all_results: if True, return the correlation between model and human ratings for every chord variants. Default=False
    
    Outputs:
    results: dataframe of the correlation between model and human ratings for  every chord variants. 
    mean_correlation: Average correlation between model and human ratings across all chord variants. 
    
    '''
    from scipy import stats
    #NOTE: THIS INCLUDES NO TIME DILATION. PERFORMANCE MAY IMPROVE IF AVERAGING MODEL PERFORMANCE ACROSS SEVERAL TIME DILATION FACTORS.
    
    if human_ranking_data_loc==None:
        human_ranking_data_loc = PATH_TO_BENCHMARK_SUITE + 'chord_alignment/Lokyan_t_03_human_processed_ratings.csv'
        
    human_results_t03 = pd.read_csv(human_ranking_data_loc, index_col = 0)
    
    chord_rating_path = PATH_TO_BENCHMARK_SUITE + 'chord_alignment/'
    
    results_human = pd.DataFrame(human_results_t03[['mean_ratings']].values.flatten(), index = human_results_t03['full_chord_names'].values.flatten(), columns = ['mean_ratings']) #{key: None for key in human_results_t03['full_chord_names'].values}#dict.fromkeys(human_results_t03['full_chord_names'].values)

    results_model = pd.DataFrame(index = human_results_t03['full_chord_names'].values.flatten(), columns = ['model_mean_NLL'])#['model_mean_NLL'] = [np.nan]*len(results)

    for x in human_results_t03['full_chord_names'].values:

        fname = chord_rating_path + x + '.mid'

        nll = get_mean_nll(fname)

        results_model.at[x, 'model_mean_NLL'] = nll
        
    results_all = results_model.copy()
    results_all['human_ratings'] = results_human['mean_ratings']
    
    correlation = stats.pearsonr(results_all['human_ratings'].values.flatten().astype(float), results_all['model_mean_NLL'].values.flatten().astype(float))[0]
    
    if return_all_results == True:
        return results_all, correlation
    
    else:
        return mean_correlation

In [31]:
# HUMAN T03 - harmony ratings

In [32]:
human_results_t03 = pd.read_csv('/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/human_data//Lokyan_t_03_human_processed_ratings.csv', index_col = 0)
chord_rating_path = '/Users/IsaacSebenius/Dropbox/CambridgePhD/ExpectationProject/CognitiveBenchmarking/data/human_data/Stimuli/t_03/'


In [33]:
results_human = pd.DataFrame(human_results_t03[['mean_ratings']].values.flatten(), index = human_results_t03['full_chord_names'].values.flatten(), columns = ['mean_ratings']) #{key: None for key in human_results_t03['full_chord_names'].values}#dict.fromkeys(human_results_t03['full_chord_names'].values)

time_dilate_marginalization = [0.5,0.75,1,1.5,2]

results_model = pd.DataFrame(index = human_results_t03['full_chord_names'].values.flatten(), columns = ['model_mean_NLL_dilation_' + str(x) for x in time_dilate_marginalization])#['model_mean_NLL'] = [np.nan]*len(results)

for x in human_results_t03['full_chord_names'].values:
    #nll = 0
    print(x)
    for dilation in time_dilate_marginalization:
        
        test_file = chord_rating_path + x + '.mid'
        test_file = time_dilate_midi(input_path=test_file, output_path = 'temp.mid', scale_factor=dilation)
        tokenizer = TSDTokenizer()
        temp_nll = get_mean_nll(lstm, TSDTokenizer(), test_file)
        #tokens = tokenizer.tokenize(test_file)
        #predictions = lstm.predict_sequence(tokens)
        #temp_nll = np.mean([nll for nll in predictions.nlls if nll is not None])
        results_model.at[x, 'model_mean_NLL_dilation_' + str(dilation)] = temp_nll #(temp_nll)
    #nll /= len(time_dilate_marginalization)
    #results[x] = [nll]
    #results.at[x, 'model_mean_NLL'] = nll

bdim_diminished
a_minor
G_major
F_major
e_minor
d_minor
C_major
C#_major
D_major
D#_major
E_major
F#_major
Ab_major
A_major
Bb_major
B_major
c_minor
c#_minor
d#_minor
f_minor
f#_minor
g_minor
ab_minor
bb_minor
b_minor
C7_seventh
C#7_seventh
D7_seventh
D#7_seventh
E7_seventh
F7_seventh
F#7_seventh
G7_seventh
Ab7_seventh
A7_seventh
Bb7_seventh
B7_seventh
c7_minor_seventh
c#7_minor_seventh
d7_minor_seventh
eb7_minor_seventh
e7_minor_seventh
f7_minor_seventh
f#7_minor_seventh
g7_minor_seventh
ab7_minor_seventh
a7_minor_seventh
bb7_minor_seventh
b7_minor_seventh


In [36]:
results_model = ((results_model - results_model.mean())/results_model.std())#.mean(1)

In [37]:
stats.spearmanr(results_model.mean(1).values.flatten().astype(float), results_human['mean_ratings'].values.flatten())

SignificanceResult(statistic=0.465204081632653, pvalue=0.0007571889349679405)

In [45]:
stats.spearmanr(results_model['model_mean_NLL_dilation_1.5'].values.flatten().astype(float), results_human['mean_ratings'].values.flatten())

SignificanceResult(statistic=0.14795918367346936, pvalue=0.3103085094179323)

In [ ]:
# Glass comparison

In [252]:
def run_glass_benchmark(times, surprisals, PATH_TO_BENCHMARK_SUITE, human_ratings_loc=None,  return_all_results = False):
    
    '''
    Calculates human-model correlation between surprisal at various points in Glass's The Hours (from glass.mid in the benchmark_suite/glass/ directory).
    
    Arguments:
    times: np.array, vector of timestamps (in seconds) where model surprisal is estimated. The more fine-grained the time is (e.g. the more samples), closer they can be aligned to the timepoints of human ratings. 
    surprisals: np.array, vector of model surprisal estimates
    PATH_TO_BENCHMARK_SUITE: str, path to benchmark_suite directory.
    human_ratings_loc: str, the location of the file "GlassEvents.txt" By default this is in the "glass" folder within the benchmark_suite directory. 
    return_all_results: Bool, whether to return the surprisal at each of the musical events where there are human ratings. Otherwise just return the correlation. Default = False
    
    
    Outputs:
    results: dataframe of the correlation between model and human ratings for  every chord variants. 
    mean_correlation: Average correlation between model and human ratings across all chord variants. 
    
    '''
        
    if human_ratings_loc==None:
        human_ratings_loc = PATH_TO_BENCHMARK_SUITE + 'glass/GlassEvents.txt'
        
    surprise_events = pd.read_csv(human_ratings_loc)
    surprise_events['Group'] = ['HS']*17 + ['LS']*17 + ['US']*17
    surprise_events = surprise_events.fillna(0)
    
    def find_nearest(array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        return idx, array[idx]
    
    model_surprisal = [surprisals[find_nearest(times, onset)[0]] for onset in surprise_events['Onset'].values.flatten()]
    surprise_events['Model surprisal'] = model_surprisal
    
    correlation = stats.spearmanr(surprise_events['Rank'], model_surprisal)[0]
    
        
    if return_all_results == True:
        return surprise_events, correlation
    
    else:
        return correlation

In [253]:
# Mussorgsky comparison


In [255]:
def run_mussosgsky_benchmark(times, surprisals, PATH_TO_BENCHMARK_SUITE, human_ratings_loc=None,  return_all_results = False):
    
    '''
    Calculates human-model correlation between surprisal at various points in Mussorgsky's Night on Bald Mountain (from nobm.mid in the benchmark_suite/mussorgsky/ directory).
    
    Arguments:
    times: np.array, vector of timestamps (in seconds) where model surprisal is estimated. The more fine-grained the time is (e.g. the more samples), closer they can be aligned to the timepoints of human ratings. 
    surprisals: np.array, vector of model surprisal estimates.
    PATH_TO_BENCHMARK_SUITE: str, path to benchmark_suite directory.
    human_ratings_loc: str, the location of the file "MussorgskyEvents.txt" By default this is in the "mussorgsky" folder within the benchmark_suite directory. 
    return_all_results: Bool, whether to return the surprisal at each of the musical events where there are human ratings. Otherwise just return the correlation. Default = False
    
    
    Outputs:
    results: dataframe of the correlation between model and human ratings for  every chord variants. 
    mean_correlation: Average correlation between model and human ratings across all chord variants. 
    
    '''
    
    if human_ratings_loc==None:
        human_ratings_loc = PATH_TO_BENCHMARK_SUITE + 'mussorgsky/MussorgskyEvents.txt'
        
    surprise_events = pd.read_csv(human_ratings_loc)
    surprise_events['Group'] = ['HS']*28 + ['LS']*28 + ['US']*28
    surprise_events = surprise_events.fillna(0)
    
    def find_nearest(array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        return idx, array[idx]
    
    model_surprisal = [surprisals[find_nearest(times, onset)[0]] for onset in surprise_events['Onset'].values.flatten()]
    surprise_events['Model surprisal'] = model_surprisal
    
    correlation = stats.spearmanr(surprise_events['Rank'], model_surprisal)[0]
    
        
    if return_all_results == True:
        return surprise_events, correlation
    
    else:
        return correlation

In [257]:
glass = pd.read_csv('../NLL_over_time_Glass_MT.csv', index_col = 0)
times, surprisal = glass['Time (s)'].values.flatten(),  glass['NLL-moving sum over 1s window'].values.flatten()


In [258]:
run_glass_benchmark(times, surprisal, PATH_TO_BENCHMARK_SUITE)

0.5053330096541302

In [259]:
mussorgsky = pd.read_csv('../NLL_over_time_Mussorgsky_MT.csv', index_col = 0)
times, surprisal = mussorgsky['Time (s)'].values.flatten(),  mussorgsky['NLL-moving sum over 1s window'].values.flatten()


In [260]:
run_mussosgsky_benchmark(times, surprisal, PATH_TO_BENCHMARK_SUITE)

0.4367548635254413